In [1]:
from pyspark import SparkContext

In [2]:
sc=SparkContext()

In [3]:
import numpy as np

1.) DSE

Write code to estimate the percentage of males and females from an RDD containing millions of such records 

In [4]:
genderList=np.random.choice(['m','f'], 10000000, p=[0.6,0.4])
genderRDD=sc.parallelize(genderList)

#Your Answer Here
sampledRDD=genderRDD.sample(withReplacement=False, fraction = 0.001)
M,F=sampledRDD.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).collect()
T=M[1]+F[1]
print 'Male Percentage = %f'%((M[1]*1.0)/T)
print 'Female Percentage = %f'%((F[1]*1.0)/T)

Male Percentage = 0.592306
Female Percentage = 0.407694


2.) CSE and DSE

Given a directed graph in the form of an RDD where each element `(i,j)` of the graphRDD represents an edge in the graph from node i to node j. Find one of the nodes with maximum out-degree in the graph and print the node and the out-degree. 

In [5]:
nodes=range(1000)
edges=set()
for e in range(100000):
    i,j=np.random.choice(nodes,2)
    if i != j:
        edges.add((i,j))
graphRDD=sc.parallelize(list(edges))

#Your answer here
degreeRDD=graphRDD.map(lambda (i,j): (i,1)).reduceByKey(lambda x,y: x+y)
N,D=degreeRDD.reduce(lambda x,y: x if x[1]>=y[1] else y)
print 'Node %d has max degree %d'%(N,D)

Node 616 has max degree 129


3.) 

Suppose that we have convert the above directed graph into an undirected graph by considering every pair of edge `(i,j)` and `(j,i)` as the same edge. Count the number of edges in this resulting undirected graph given the directed graph in graphRDD.

In [6]:
#Your answer here
undirectedRDD=graphRDD.map(lambda (i,j): (i,j) if i<j else (j,i))
edgeRDD=undirectedRDD.distinct()
print 'Number of edges = %d'%edgeRDD.count()

Number of edges = 90626


4.)

Coming back to the directed graph represented in graphRDD, calculate the number of non-reciprocative relations between nodes in the graph i.e there is an edge `(i,j)` but no edge `(j,i)` in the graph. Also report the number of reciprocative relations.

In [7]:
#Your answer here
undirectedRDD=graphRDD.map(lambda (i,j): (i,j) if i<j else (j,i)).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
unrec,rec=undirectedRDD.map(lambda (e,c): (c,1)).reduceByKey(lambda x,y: x+y).collect()
print 'Number of un-reciprocated relations = %d'%(unrec[1])
print 'Number of reciprocated relations = %d'%(rec[1])

Number of un-reciprocated relations = 86134
Number of reciprocated relations = 4492


5.) CSE and DSE

Given an RDD containing details of customers and another RDD containing details of their purchases, generate the following purchase report in the form of an RDD.

`
[('C3', (('f', 'AZ'), '4')),
 ('C2', (('f', 'CA'), None)),
 ('C1', (('m', 'CA'), '2')),
 ('C5', (('m', 'NJ'), None)),
 ('C4', (('m', 'NY'), '7'))]
`

In [8]:
customerRDD=sc.parallelize([('C1',('m','CA')),('C2',('f','CA')),('C3',('f','AZ')),('C4',('m','NY')),('C5',('m','NJ'))])
purchaseRDD=sc.parallelize([('C1','2'),('C3','4'),('C4','7')])

#Your answer here
reportRDD=customerRDD.leftOuterJoin(purchaseRDD)
reportRDD.collect()

[('C3', (('f', 'AZ'), '4')),
 ('C2', (('f', 'CA'), None)),
 ('C1', (('m', 'CA'), '2')),
 ('C5', (('m', 'NJ'), None)),
 ('C4', (('m', 'NY'), '7'))]

6.)

mathRDD contains the list of students who have taken the mathematics course and psyRDD contains the list of students who have taken the psychology course. Count the number of students who have taken either of the courses but not both.

In [9]:
mathRDD=sc.parallelize(np.random.choice(range(50),25,replace=False))
psyRDD=sc.parallelize(np.random.choice(range(50),30,replace=False))

#Your answer here
M=mathRDD.subtract(psyRDD).count()
P=psyRDD.subtract(mathRDD).count()
print 'Number of students who have taken either courses but not both = %d'%(M+P)

Number of students who have taken either courses but not both = 29
